# Interacting with JetClass-II and the Sophon model

This is a self-contained notebook that

 - explores the JetClass-II dataset, and visualizes jet samples as point clouds
 - infers the Sophon model via ONNX in Python and inspects its output nodes  

If you are looking for a C++ n-tuplizer to proceed Delphes ROOT files and integrate the Sophon model, refer to [this section](https://github.com/jet-universe/sophon/tree/main?tab=readme-ov-file#c-workflow-for-analyzing-delphes-files).

This notebook is designed to [run on Google Colab](https://colab.research.google.com/github/jet-universe/sophon/blob/main/notebooks/Interacting_with_JetClassII_and_Sophon.ipynb).

## Install dependencies

In [126]:
! pip install weaver-core
! pip install onnxruntime
#! pip install torch torchvision torchaudio # google colab has torch installed

# Exploring JetClass-II

Load a few entries from JetClass-II as an example and display their data structure.

In [127]:
! wget --no-verbose https://github.com/jet-universe/sophon/raw/main/notebooks/JetClassII_example.parquet

2025-07-14 22:23:29 URL:https://raw.githubusercontent.com/jet-universe/sophon/main/notebooks/JetClassII_example.parquet [447746/447746] -> "JetClassII_example.parquet.4" [1]


In [128]:
import awkward as ak

arrays = ak.from_parquet("JetClassII_example.parquet")
arrays

<Array [{part_px: [...], ...}, ..., {...}] type='100 * {part_px: var * floa...'>

The entries are jet-based. Each entry includes

 - the jet features (`jet_*`),
 - the jet constituent's features (`part_*`),
 - the GEN-jet features (`genjet_*`) if a matched GEN-jet is found, and
 - the GEN-jet constituent's features (`genpart_*`).
 - auxiliary features for selected truth particles (`aux_genpart_*`)

Display the content of each branch.

In [129]:
for branch in arrays.fields:
    print(branch, ': ', arrays[branch])

part_px :  [[-202, -140, -76.9, -49.3, -44.9, ..., -0.866, -0.597, -0.563, -0.491], ...]
part_py :  [[29, -104, -2.95, -1.38, -1.01, ..., -0.175, 0.213, 0.322, 0.19, -0.253], ...]
part_pz :  [[-44, -65.4, -54.7, -34.4, -32.4, ..., -0.474, -0.289, 0.0158, -0.752], ...]
part_energy :  [[209, 186, 94.5, 60.2, 55.4, 51.3, ..., 0.954, 1.01, 0.737, 0.594, 0.933], ...]
part_deta :  [[-0.232, -0.0793, 0.215, 0.204, ..., 0.0635, -0.0315, -0.473, 0.669], ...]
part_dphi :  [[-0.251, 0.529, -0.07, -0.0804, ..., -0.349, -0.604, -0.434, 0.368], ...]
part_d0val :  [[0.00359, -0.0036, -0.0101, 0.0104, 0, -0.00337, ..., 0, 0, 0, 0, 0], ...]
part_d0err :  [[0.011, 0.011, 0.011, 0.0116, 0, 0.0116, ..., 0.0926, 0, 0, 0, 0, 0], ...]
part_dzval :  [[-0.0224, 0.0094, -0.00443, -0.0521, 0, -0.00956, ..., 0, 0, 0, 0, 0], ...]
part_dzerr :  [[0.0318, 0.0318, 0.0316, 0.0316, 0, 0.0348, ..., 0.198, 0, 0, 0, 0, 0], ...]
part_charge :  [[1, -1, 1, -1, 0, -1, 1, -1, 1, 1, 1, ..., 0, 0, 0, 0, -1, 0, 0, 0, 0, 0], ...]

# Visualizing JetClass-II

Let's visualize the jet and the matched GEN-jet as point clouds.

In [130]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np

label_list = ["label_X_bb", "label_X_cc", "label_X_ss", "label_X_qq", "label_X_bc", "label_X_cs", "label_X_bq", "label_X_cq", "label_X_sq", "label_X_gg", "label_X_ee", "label_X_mm", "label_X_tauhtaue", "label_X_tauhtaum", "label_X_tauhtauh", "label_X_YY_bbbb", "label_X_YY_bbcc", "label_X_YY_bbss", "label_X_YY_bbqq", "label_X_YY_bbgg", "label_X_YY_bbee", "label_X_YY_bbmm", "label_X_YY_bbtauhtaue", "label_X_YY_bbtauhtaum", "label_X_YY_bbtauhtauh", "label_X_YY_bbb", "label_X_YY_bbc", "label_X_YY_bbs", "label_X_YY_bbq", "label_X_YY_bbg", "label_X_YY_bbe", "label_X_YY_bbm", "label_X_YY_cccc", "label_X_YY_ccss", "label_X_YY_ccqq", "label_X_YY_ccgg", "label_X_YY_ccee", "label_X_YY_ccmm", "label_X_YY_cctauhtaue", "label_X_YY_cctauhtaum", "label_X_YY_cctauhtauh", "label_X_YY_ccb", "label_X_YY_ccc", "label_X_YY_ccs", "label_X_YY_ccq", "label_X_YY_ccg", "label_X_YY_cce", "label_X_YY_ccm", "label_X_YY_ssss", "label_X_YY_ssqq", "label_X_YY_ssgg", "label_X_YY_ssee", "label_X_YY_ssmm", "label_X_YY_sstauhtaue", "label_X_YY_sstauhtaum", "label_X_YY_sstauhtauh", "label_X_YY_ssb", "label_X_YY_ssc", "label_X_YY_sss", "label_X_YY_ssq", "label_X_YY_ssg", "label_X_YY_sse", "label_X_YY_ssm", "label_X_YY_qqqq", "label_X_YY_qqgg", "label_X_YY_qqee", "label_X_YY_qqmm", "label_X_YY_qqtauhtaue", "label_X_YY_qqtauhtaum", "label_X_YY_qqtauhtauh", "label_X_YY_qqb", "label_X_YY_qqc", "label_X_YY_qqs", "label_X_YY_qqq", "label_X_YY_qqg", "label_X_YY_qqe", "label_X_YY_qqm", "label_X_YY_gggg", "label_X_YY_ggee", "label_X_YY_ggmm", "label_X_YY_ggtauhtaue", "label_X_YY_ggtauhtaum", "label_X_YY_ggtauhtauh", "label_X_YY_ggb", "label_X_YY_ggc", "label_X_YY_ggs", "label_X_YY_ggq", "label_X_YY_ggg", "label_X_YY_gge", "label_X_YY_ggm", "label_X_YY_bee", "label_X_YY_cee", "label_X_YY_see", "label_X_YY_qee", "label_X_YY_gee", "label_X_YY_bmm", "label_X_YY_cmm", "label_X_YY_smm", "label_X_YY_qmm", "label_X_YY_gmm", "label_X_YY_btauhtaue", "label_X_YY_ctauhtaue", "label_X_YY_stauhtaue", "label_X_YY_qtauhtaue", "label_X_YY_gtauhtaue", "label_X_YY_btauhtaum", "label_X_YY_ctauhtaum", "label_X_YY_stauhtaum", "label_X_YY_qtauhtaum", "label_X_YY_gtauhtaum", "label_X_YY_btauhtauh", "label_X_YY_ctauhtauh", "label_X_YY_stauhtauh", "label_X_YY_qtauhtauh", "label_X_YY_gtauhtauh", "label_X_YY_qqqb", "label_X_YY_qqqc", "label_X_YY_qqqs", "label_X_YY_bbcq", "label_X_YY_ccbs", "label_X_YY_ccbq", "label_X_YY_ccsq", "label_X_YY_sscq", "label_X_YY_qqbc", "label_X_YY_qqbs", "label_X_YY_qqcs", "label_X_YY_bcsq", "label_X_YY_bcs", "label_X_YY_bcq", "label_X_YY_bsq", "label_X_YY_csq", "label_X_YY_bcev", "label_X_YY_csev", "label_X_YY_bqev", "label_X_YY_cqev", "label_X_YY_sqev", "label_X_YY_qqev", "label_X_YY_bcmv", "label_X_YY_csmv", "label_X_YY_bqmv", "label_X_YY_cqmv", "label_X_YY_sqmv", "label_X_YY_qqmv", "label_X_YY_bctauev", "label_X_YY_cstauev", "label_X_YY_bqtauev", "label_X_YY_cqtauev", "label_X_YY_sqtauev", "label_X_YY_qqtauev", "label_X_YY_bctaumv", "label_X_YY_cstaumv", "label_X_YY_bqtaumv", "label_X_YY_cqtaumv", "label_X_YY_sqtaumv", "label_X_YY_qqtaumv", "label_X_YY_bctauhv", "label_X_YY_cstauhv", "label_X_YY_bqtauhv", "label_X_YY_cqtauhv", "label_X_YY_sqtauhv", "label_X_YY_qqtauhv", "label_QCD_bbccss", "label_QCD_bbccs", "label_QCD_bbcc", "label_QCD_bbcss", "label_QCD_bbcs", "label_QCD_bbc", "label_QCD_bbss", "label_QCD_bbs", "label_QCD_bb", "label_QCD_bccss", "label_QCD_bccs", "label_QCD_bcc", "label_QCD_bcss", "label_QCD_bcs", "label_QCD_bc", "label_QCD_bss", "label_QCD_bs", "label_QCD_b", "label_QCD_ccss", "label_QCD_ccs", "label_QCD_cc", "label_QCD_css", "label_QCD_cs", "label_QCD_c", "label_QCD_ss", "label_QCD_s", "label_QCD_light"]

def color_fader(c1, c2, mix=0): # linear interpolate from color c1 (mix=0) to c2 (mix=1)
    assert 0 <= mix <= 1
    c1 = np.array(mpl.colors.to_rgb(c1))
    c2 = np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

def visualize_jet(ax, arr, is_genjet=True):
    # part_type: 0: electron, 1: muon, 2: photon, 3: charged had, 4: neutral had
    if not is_genjet:
        jet_prefix, part_prefix = 'jet', 'part'
        part_disp = np.hypot(arr['part_d0val'], arr['part_dzval']) # an estimation of displacement
        part_type = ak.argmax([
            arr['part_isElectron'],
            arr['part_isMuon'],
            arr['part_isPhoton'],
            arr['part_isChargedHadron'],
            arr['part_isNeutralHadron']
        ], axis=0)
    else:
        jet_prefix, part_prefix = 'genjet', 'genpart'
        part_disp = np.sqrt(arr['genpart_x']**2 + arr['genpart_y']**2 + arr['genpart_z']**2)
        part_type = ak.argmax([
            abs(arr['genpart_pid']) == 11,
            abs(arr['genpart_pid']) == 13,
            arr['genpart_pid'] == 22,
            (abs(arr['genpart_pid']) == 211) | (abs(arr['genpart_pid']) == 321) | (abs(arr['genpart_pid']) == 2212),
            (arr['genpart_pid'] == 130) | (arr['genpart_pid'] == 310) | (arr['genpart_pid'] == 2112)
        ], axis=0)
    radius_fn = lambda pt: np.sqrt(pt) / 200

    # draw particles
    for i in range(arr[f"{jet_prefix}_nparticles"]):
        if not is_genjet:
            px, py, deta, dphi = map(lambda x: arr[f"part_{x}"][i], ['px', 'py', 'deta', 'dphi'])
        else:
            px, py, deta, dphi = map(lambda x: arr[f"genpart_{x}"][i], ['px', 'py', 'jet_deta', 'jet_dphi'])
        pt = np.hypot(px, py)

        color = color_fader('#74c476', '#081d58', mix=np.tanh(part_disp[i]))
        if part_type[i] == 0: # electrons
            ax.add_patch(mpl.patches.RegularPolygon((deta, dphi), 3, radius=radius_fn(pt), clip_on=True, alpha=0.3, edgecolor='black', facecolor=color))
        elif part_type[i] == 1: # muons
            ax.add_patch(mpl.patches.RegularPolygon((deta, dphi), 3, radius=radius_fn(pt), orientation=np.pi, clip_on=True, alpha=0.3, edgecolor='black', facecolor=color))
        elif part_type[i] == 2: # photons
            ax.add_patch(mpl.patches.RegularPolygon((deta, dphi), 5, radius=radius_fn(pt), clip_on=True, alpha=0.3, edgecolor=color, facecolor='none'))
        elif part_type[i] == 3: # charged hadrons
            ax.add_patch(plt.Circle((deta, dphi), radius_fn(pt), clip_on=True, alpha=0.3, facecolor=color))
        elif part_type[i] == 4: # neutral hadrons
            ax.add_patch(plt.Circle((deta, dphi), radius_fn(pt), clip_on=True, alpha=0.3, edgecolor=color, facecolor='none'))

        # if is_genjet:
        #     ax.text(deta, dphi, str(arr.genpart_pid[i]), ha='center', va='center', fontsize=4) # annotate particle PDGIDs

    # auxiliary truth particles if this is a genjet
    if is_genjet:
        for i in range(len(arr["aux_genpart_pt"])):
            color = 'yellow' if arr.aux_genpart_isResX[i] else 'gold' if arr.aux_genpart_isResY[i] else 'orange' if arr.aux_genpart_isResDecayProd[i] else None
            if color is None:
                continue
            pt = arr.aux_genpart_pt[i]
            deta = np.sign(arr.jet_eta) * (arr.aux_genpart_eta[i] - arr.jet_eta)
            dphi = (arr.aux_genpart_phi[i] - arr.jet_phi + np.pi) % (2*np.pi) - np.pi
            pid = arr.aux_genpart_pid[i]
            if abs(deta) > 0.8 or abs(dphi) > 0.8:
                continue

            # draw squares for auxiliary particles
            ax.add_patch(mpl.patches.RegularPolygon((deta, dphi), 4, radius=radius_fn(pt), clip_on=True, alpha=0.2, facecolor=color))
            ax.text(deta, dphi, str(pid), ha='center', va='center', fontsize=5) # annotate particle PDGIDs

    ax.set_xlabel(r'$\Delta\eta$'); ax.set_ylabel(r'$\Delta\phi$')
    ax.set_xlim(-0.8, 0.8); ax.set_ylim(-0.8, 0.8)
    ax.set_aspect('equal')

    # annotate basic info
    ax.text(0.02, 0.98, r'$\mathbf{%s}$ $p_{\rm T}$: %.0f GeV, $m_{\rm SD}$: %.0f GeV' % (jet_prefix, arr[f"{jet_prefix}_pt"], arr[f"{jet_prefix}_sdmass"]), ha='left', va='top', transform=ax.transAxes)
    if not is_genjet:
        ax.text(0.02, 0.91, r'truth label: %s' % label_list[arr[f"{jet_prefix}_label"]][6:], ha='left', va='top', transform=ax.transAxes)

Left subplot: jet, right subplot: matched GEN-jet

Note:
 - Hadrons = **circles**; electrons/muons = **triangles** (upward/downward); photons = **pentagons**.
 - Charged particles = **solid** markers; neutral particles = **hollow** markers.
 - Larger displacement -> more **blue**.

For GEN-jet:
 - the GEN resonances (X, Y) = **(yellow, gold) squares**; decay products = **orange squares**, PDGIDs annotated.

In [131]:
i = 1  # draw entry number i

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
visualize_jet(ax1, arrays[i], is_genjet=False)
visualize_jet(ax2, arrays[i], is_genjet=True)
plt.show()

# Inferring Sophon model

Download the Sophon ONNX model from Hugging Face.

In [132]:
! wget --no-verbose https://huggingface.co/jet-universe/sophon/resolve/main/models/JetClassII_Sophon/model.onnx -O JetClassII_Sophon.onnx

2025-07-14 22:23:31 URL:https://cdn-lfs-us-1.hf.co/repos/4a/b9/4ab947e5de3182c3f0fe6225553fb30b989aabcea1ec7f653a4c814f62f36c41/f54e9fd56b12e22ef4266e40b683dbf8533139b55cdab1b1c127d6e7cb8e2fd4?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.onnx%3B+filename%3D%22model.onnx%22%3B&Expires=1752535411&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MjUzNTQxMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzRhL2I5LzRhYjk0N2U1ZGUzMTgyYzNmMGZlNjIyNTU1M2ZiMzBiOTg5YWFiY2VhMWVjN2Y2NTNhNGM4MTRmNjJmMzZjNDEvZjU0ZTlmZDU2YjEyZTIyZWY0MjY2ZTQwYjY4M2RiZjg1MzMxMzliNTVjZGFiMWIxYzEyN2Q2ZTdjYjhlMmZkND9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=HcUbbXe2X5mpEL1nchiGTz7bRFk1CI5lUersVHywceHTl3H3bq0DwrXen4DdT1cRpy7x-NRn8yK2aRKFoxA9NRv8y%7EckIcoEAbZXFx1JbLOcP8xz7qD3pAGJPvNFs9JeErlUTfSb%7EJNbEo0A48pA4IZ5bBEfTW4fStD0ZHqEd5HaZTJr53UPA6AR0DZ4TdaaSFoHcahJQncPbMKKAxN82xm0p6A-89SDHPpii9uFuuHaEdclbAiDdPv6ESIIYXrnHIgw7PIO1-Wz3ZaThFd

Start an ONNX Runtime session.

In [133]:
import onnxruntime as ort

ort_sess = ort.InferenceSession('JetClassII_Sophon.onnx')

Make input variables and do inference.

In [134]:
def make_input_feats(arr):
    # calculate all input features to the Sophon model
    input_feats = {}

    # existing variables
    for var in ['part_deta', 'part_dphi', 'part_charge', 'part_d0err', 'part_dzerr', 'part_isElectron', 'part_isMuon', 'part_isPhoton', 'part_isChargedHadron', 'part_isNeutralHadron']:
        input_feats[var] = ak.values_astype(arr[var], np.float32)
    # new variables
    input_feats['part_mask'] = ak.ones_like(arr.part_px)
    input_feats['part_px_scale'] = arr.part_px / arr.jet_pt * 500
    input_feats['part_py_scale'] = arr.part_py / arr.jet_pt * 500
    input_feats['part_pz_scale'] = arr.part_pz / arr.jet_pt * 500
    input_feats['part_energy_scale'] = arr.part_energy / arr.jet_pt * 500
    input_feats['part_pt'] = np.hypot(arr.part_px, arr.part_py)
    input_feats['part_pt_scale'] = np.hypot(input_feats['part_px_scale'], input_feats['part_py_scale'])
    input_feats['part_pt_scale_log'] = np.log(input_feats['part_pt_scale'])
    input_feats['part_e_scale_log'] = np.log(input_feats['part_energy_scale'])
    input_feats['part_logptrel'] = np.log(input_feats['part_pt'] / arr.jet_pt)
    input_feats['part_logerel'] = np.log(arr.part_energy / arr.jet_energy)
    input_feats['part_deltaR'] = np.hypot(arr.part_deta, arr.part_dphi)
    input_feats['part_d0'] = np.tanh(arr.part_d0val)
    input_feats['part_dz'] = np.tanh(arr.part_dzval)

    return input_feats


def infer_model(input_feats, debug=False):
    # define data structure and config
    input_names = ["pf_features", "pf_vectors", "pf_mask"]
    input_shapes = [(1, 17, 128), (1, 4, 128), (1, 1, 128)] # (batch_size=1, channel, length)
    input_var_info = [ # (name, subtract_val, multiply_val, clip_min, clip_max)
        [
            ("part_pt_scale_log", 1.7, 0.7, -5, 5),
            ("part_e_scale_log", 2.0, 0.7, -5, 5),
            ("part_logptrel", -4.7, 0.7, -5, 5),
            ("part_logerel", -4.7, 0.7, -5, 5),
            ("part_deltaR", 0.2, 4.0, -5, 5),
            ("part_charge", 0, 1, -1e8, 1e8),
            ("part_isChargedHadron", 0, 1, -1e8, 1e8),
            ("part_isNeutralHadron", 0, 1, -1e8, 1e8),
            ("part_isPhoton", 0, 1, -1e8, 1e8),
            ("part_isElectron", 0, 1, -1e8, 1e8),
            ("part_isMuon", 0, 1, -1e8, 1e8),
            ("part_d0", 0, 1, -1e8, 1e8),
            ("part_d0err", 0, 1, 0, 1),
            ("part_dz", 0, 1, -1e8, 1e8),
            ("part_dzerr", 0, 1, 0, 1),
            ("part_deta", 0, 1, -1e8, 1e8),
            ("part_dphi", 0, 1, -1e8, 1e8),
        ],
        [
            ("part_px_scale", 0, 1, -1e8, 1e8),
            ("part_py_scale", 0, 1, -1e8, 1e8),
            ("part_pz_scale", 0, 1, -1e8, 1e8),
            ("part_energy_scale", 0, 1, -1e8, 1e8),
        ],
        [
            ("part_mask", 0, 1, -1e8, 1e8),
        ],
    ]

    input = {name: [] for name in input_names}
    for name, shape, var_info in zip(input_names, input_shapes, input_var_info):
        if (debug):
            print("====== Preprocessed input: ", name, shape, "======")
        length = shape[-1]
        for var, sub, mul, clip_min, clip_max in var_info:
            v = ak.to_numpy(input_feats[var])
            v = (v - sub) * mul  # variable shifting and scaling
            v = np.pad(v, (0, length))[:length] # zero-padding to fixed length
            v = np.clip(v, clip_min, clip_max)  # clipping
            input[name].append(v)
            if (debug):
                print(' - ', var, v)

        input[name] = np.stack(input[name]).reshape(shape)

    # run inference
    ort_out = ort_sess.run(None, input)
    return ort_out[0][0]


In [135]:
i = 1  # infer jet entry number i

input_feats = make_input_feats(arrays[i])
output = infer_model(input_feats, debug=False)
probs, hidneurons = output[:188], output[188:]

print('Sophon model scores (len=188):', probs[:5], '...')
print('Sophon model latent features (len=128):', hidneurons[:5], '...')

Sophon model scores (len=188): [0.00087492 0.00014806 0.00021878 0.00013643 0.00019383] ...
Sophon model latent features (len=128): [-0.00624361  0.00927145 -0.02239747 -0.00590648 -0.01222859] ...


# Visualizing Sophon's output probabilities

Given a jet entry, find the top-5 probabilities interpreted by the Sophon model, and visualize these probabilities alongside the jet's visualization.

In [136]:
def visualize_top5_probs(ax, probs):
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    axin = inset_axes(ax, width="20%", height="20%", loc='lower right')

    top_inds = np.argsort(probs)[::-1][:5]
    axin.barh([label_list[i][6:] for i in top_inds], probs[top_inds], color='skyblue')
    axin.invert_yaxis()
    axin.xaxis.tick_top()
    axin.tick_params(axis='both', which='major', labelsize=6)
    axin.set_title('Sophon\'s\nTop-5 probs', fontsize=8)

In [137]:
i = 0  # draw entry number i

# infer scores
input_feats = make_input_feats(arrays[i])
probs = infer_model(input_feats, debug=False)[:188]

# make plots
f, ax = plt.subplots(1, 1, figsize=(5, 5))
visualize_jet(ax, arrays[i], is_genjet=False)
visualize_top5_probs(ax, probs)

plt.show()

In [138]:
!ls

example_ParticleTransformer_sophon.py  JetClassII_example.parquet.4
JetClassII_example.parquet	       JetClassII_Sophon.onnx
JetClassII_example.parquet.1	       __pycache__
JetClassII_example.parquet.2	       sample_data
JetClassII_example.parquet.3


In [168]:
!git clone https://github.com/hqucms/weaver-core.git

Cloning into 'weaver-core'...
remote: Enumerating objects: 977, done.
remote: Counting objects: 100% (438/438), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 977 (delta 403), reused 331 (delta 331), pack-reused 539 (from 2)
Receiving objects: 100% (977/977), 246.74 KiB | 6.33 MiB/s, done.
Resolving deltas: 100% (660/660), done.


In [169]:
from google.colab import files

uploaded = files.upload()


Saving example_ParticleTransformer_sophon.py to example_ParticleTransformer_sophon (2).py


In [170]:
from example_ParticleTransformer_sophon import get_model


In [171]:
# kinematic features (vectors)
px = arrays['part_px']
py = arrays['part_py']
pz = arrays['part_pz']
penergy = arrays['part_energy']
# turn into a nested awkward array(dictionary)
knmtc_points = ak.zip({
    "px": px,
    "py": py,
    "pz": pz,
    "E": penergy
}, depth_limit=1)

# padding in order to mask=mask
# according to doc ak.pad_none() "increases lengths of lists to a target length by adding None values."
knmtc_padded = ak.pad_none(knmtc_points, target=100, axis=1, clip=True)

In [172]:
knmtc_list = ak.to_list(knmtc_padded)

knmtc_cleaned = []
for jet in knmtc_list:
    if jet is None:
        continue  # skip completely empty jets
    clean_jet = []
    for p in jet:
        if p is None or not all(k in p for k in ['px', 'py', 'pz', 'E']):
            clean_jet.append([0, 0, 0, 0])
        else:
            clean_jet.append([p['px'], p['py'], p['pz'], p['E']])
    # pad to 100
    while len(clean_jet) < 100:
        clean_jet.append([0, 0, 0, 0])
    knmtc_cleaned.append(clean_jet[:100])  # truncate if necessary

# Convert to tensor
knmtc_tensor = torch.tensor(knmtc_cleaned, dtype=torch.float)
print("knmtc_tensor shape:", knmtc_tensor.shape)


knmtc_tensor shape: torch.Size([100, 100, 4])


In [173]:
# 1. Turn mask into boolean
mask_bool = mask_tensor.bool()

# 2. Extract points from Lorentz vectors (px, py, pz)
points = knmtc_tensor[:, :, :3]  # shape: [B, 100, 3]

# 3. Move everything to device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

points = points.to(device)
features_tensor = features_tensor.to(device)
knmtc_tensor = knmtc_tensor.to(device)
mask_bool = mask_bool.to(device)


In [174]:
part_features = ak.zip({
    "charge": arrays['part_charge'],
    "electron": arrays['part_isElectron'],
    "muon": arrays['part_isMuon'],
    "photon": arrays['part_isPhoton'],
    "chargedhadron": arrays['part_isChargedHadron'],
    "neutralhadron": arrays['part_isNeutralHadron']
}, depth_limit=1)

pfeats_padded = ak.pad_none(part_features, target=100, axis=1, clip=True)

pfeats_list = ak.to_list(pfeats_padded)
pfeats_cleaned = []

for jet in pfeats_list:
    if jet is None:
        continue

    n_particles = len(jet['charge'])  # number of particles (100)
    clean_jet = []

    for i in range(n_particles):
        # addressing None value errors that kept coming up...
        if any(jet[key][i] is None for key in jet):
            clean_jet.append([0]*6)
        else:
            clean_jet.append([
                jet['charge'][i],
                jet['electron'][i],
                jet['muon'][i],
                jet['photon'][i],
                jet['chargedhadron'][i],
                jet['neutralhadron'][i]
            ])

    pfeats_cleaned.append(clean_jet)

# to tensor
features_tensor = torch.tensor(pfeats_cleaned, dtype=torch.float)
print("features_tensor shape:", features_tensor.shape)


features_tensor shape: torch.Size([100, 100, 6])


In [175]:
# addressing mask
mask = ak.ones_like(arrays['part_px'], dtype=np.int32)
mask_padded = ak.pad_none(mask, target=100, axis=1, clip=True)

# mask/padding
mask_filled = ak.fill_none(mask_padded, 0)

# to tensor
mask_tensor = torch.tensor(ak.to_numpy(mask_filled), dtype=torch.int)
print("mask_tensor shape:", mask_tensor.shape)


mask_tensor shape: torch.Size([100, 100])


In [187]:
from example_ParticleTransformer_sophon import get_model
from example_ParticleTransformer_sophon import get_model
from weaver.nn.model.ParticleTransformer import ParticleTransformer
import torch
from example_ParticleTransformer_sophon import ParticleTransformerSophonWrapper
from example_ParticleTransformer_sophon import ParticleTransformerSophonWrapper

from example_ParticleTransformer_sophon import ParticleTransformerSophonWrapper

# 🧠 FIX: Let’s define a corrected DummyDataConfig and call get_model with proper arguments

class DummyDataConfig:
    input_dicts = {
        'pf_features': ['charge', 'electron', 'muon', 'photon', 'chargedhadron', 'neutralhadron']
    }
    input_names = ['points', 'features', 'lorentz_vectors', 'mask']
    input_shapes = {
        'points': (1, 100, 3),
        'features': (1, 100, 6),
        'lorentz_vectors': (1, 100, 4),
        'mask': (1, 100)
    }

data_config = DummyDataConfig()

# ✅ Specify the number of classes (fixing the issue)
model, _ = get_model(data_config, input_dim=6, num_classes=60)
model.to(device)
model.eval()  # still sets model to inference mode

# No need to load checkpoint — model has random weights



ParticleTransformerSophonWrapper(
  (mod): ParticleTransformer(
    (trimmer): SequenceTrimmer()
    (embed): Embed(
      (input_bn): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (embed): Sequential(
        (0): LayerNorm((6,), eps=1e-05, elementwise_affine=True)
        (1): Linear(in_features=6, out_features=128, bias=True)
        (2): GELU(approximate='none')
        (3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (4): Linear(in_features=128, out_features=512, bias=True)
        (5): GELU(approximate='none')
        (6): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (7): Linear(in_features=512, out_features=128, bias=True)
        (8): GELU(approximate='none')
      )
    )
    (pair_embed): PairEmbed(
      (embed): Sequential(
        (0): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
        (2): BatchNorm1d(64, e

In [189]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Use batch size of 2, number of points N = 10, feature dims from your data_config (example)
batch_size = 2
N = 10  # number of particles (can vary, just pick something small for test)
input_dim = len(data_config.input_dicts['pf_features'])  # from your data config, e.g. 6
pair_dim = 4  # from your config

# Dummy inputs - random floats, shapes must match model expectations
points = torch.randn(batch_size, N, 3).to(device)           # e.g. 3D points (x,y,z) - adjust if needed
features = torch.randn(batch_size, N, input_dim).to(device) # particle features
lorentz_vectors = torch.randn(batch_size, N, 4).to(device)  # Lorentz vector for each particle
mask = torch.zeros(batch_size, N, dtype=torch.bool).to(device)  # mask: False means valid, True means padded

# Run forward pass
model.to(device)
model.eval()

with torch.no_grad():
  output = model(points, features, lorentz_vectors, mask)


AttributeError: 'ParticleTransformer' object has no attribute '_forward_encoder'